In [350]:
import pandas as pd
import requests
import json
import base64

In [351]:
data = pd.read_csv('data/diabetes_prediction_dataset.csv')
inputs = data.iloc[2, :8].to_dict()
inputs

{'gender': 'Male',
 'age': 28.0,
 'hypertension': 0,
 'heart_disease': 0,
 'smoking_history': 'never',
 'bmi': 27.32,
 'HbA1c_level': 5.7,
 'blood_glucose_level': 158}

In [352]:
data.iloc[2, :9]

gender                  Male
age                     28.0
hypertension               0
heart_disease              0
smoking_history        never
bmi                    27.32
HbA1c_level              5.7
blood_glucose_level      158
diabetes                   0
Name: 2, dtype: object

In [353]:
def prepare_json(inputs: dict):
    feature_spec = dict()

    for keys, values in inputs.items():
        if isinstance(values, float):
            feature_spec[keys] = tf.train.Feature(float_list=tf.train.FloatList(value=[values]))
        elif isinstance(values, int):
            feature_spec[keys] = tf.train.Feature(int64_list=tf.train.Int64List(value=[values]))
        elif isinstance(values, str):
            feature_spec[keys] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[values.encode()]))

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]

    return json.dumps({
        "signature_name": "serving_default",
        "instances": result,
    })

json_data = prepare_json(inputs)

endpoint = "https://diabetes-classification-production.up.railway.app/v1/models/diabetes-classification-model:predict"
response = requests.post(endpoint, data=json_data)

prediction = response.json().get("predictions")
if prediction:
    prediction_value = prediction[0][0]
    result = "No diabetes" if prediction_value < 0.5 else "Diabetes"
else:
    result = "Error: No predictions found in response."

print(result)

No diabetes
